# ChatQwQ

This will help you getting started with QwQ [chat models](/langchain_qwq/chat_models.py). For detailed documentation of all ChatQwQ features and configurations head to the [API reference](https://python.langchain.com/api_reference/langchain-qwq/chat_models/langchain_qwq.chat_models.ChatQwQ.html).

## Overview
### Integration details


| Class | Package | Local | Serializable | Package downloads | Package latest |
| :--- | :--- | :---: |  :---: | :---: | :---: |
| [ChatQwQ](https://python.langchain.com/api_reference/langchain-qwq/chat_models/langchain_qwq.chat_models.ChatQwQ.html) | [langchain-qwq](https://python.langchain.com/api_reference/langchain-qwq/) | ❌ | beta | ![PyPI - Downloads](https://img.shields.io/pypi/dm/langchain-qwq?style=flat-square&label=%20) | ![PyPI - Version](https://img.shields.io/pypi/v/langchain-qwq?style=flat-square&label=%20) |

### Model features
| [Tool calling](/docs/how_to/tool_calling) | [Structured output](/docs/how_to/structured_output/) | JSON mode | [Image input](/docs/how_to/multimodal_inputs/) | Audio input | Video input | [Token-level streaming](/docs/how_to/chat_streaming/) | Native async | [Token usage](/docs/how_to/chat_token_usage_tracking/) | [Logprobs](/docs/how_to/logprobs/) |
| :---: | :---: | :---: | :---: |  :---: | :---: | :---: | :---: | :---: | :---: |
| ✅ | ✅ | ✅ |❌  | ❌ | ❌ | ✅ | ✅ | ✅ | ❌ | 

## Setup

To access QwQ models you'll need to create an Alibaba Cloud account, get an API key, and install the `langchain-qwq` integration package.

### Credentials

Head to [Alibaba's API Key page](https://account.alibabacloud.com/login/login.htm?oauth_callback=https%3A%2F%2Fbailian.console.alibabacloud.com%2F%3FapiKey%3D1&lang=en#/api-key) to sign up to Alibaba Cloud and generate an API key. Once you've done this set the `DASHSCOPE_API_KEY` environment variable:

In [1]:
import getpass
import os

if not os.getenv("DASHSCOPE_API_KEY"):
    os.environ["DASHSCOPE_API_KEY"] = getpass.getpass("Enter your Dashscope API key: ")

### Installation

The LangChain QwQ integration lives in the `langchain-qwq` package:

In [ ]:
%pip install -qU langchain-qwq

## Instantiation

Now we can instantiate our model object and generate chat completions:

In [2]:
from langchain_qwq import ChatQwQ

llm = ChatQwQ(
    model="qwq-plus",
    max_tokens=3_000,
    timeout=None,
    max_retries=2,
    # other params...
)

## Invocation

In [3]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French."
        "Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="J'aime la programmation.", additional_kwargs={'reasoning_content': 'Okay, the user wants me to translate "I love programming." into French. Let me start by breaking down the sentence. The subject is "I," which translates to "Je" in French. The verb is "love," which in this context is present tense, so "aime." The object is "programming," which is a noun. Now, in French, "programming" can be translated as "la programmation." \n\nWait, I should make sure that "programmation" is the right term here. Sometimes, depending on the context, it could be "programmer" as the verb, but since it\'s the noun here, "programmation" is correct. \n\nPutting it all together: "Je aime la programmation." Hmm, but in French, there\'s a common contraction for "je aime." It should be "j\'aime" to avoid the awkward "je aime" sound. So the correct sentence would be "J\'aime la programmation."\n\nI should also check if there are any other nuances. The original sentence is straightforward, so t

## Chaining

We can [chain](/docs/how_to/sequence/) our model with a prompt template like so:

In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate(
    [
        (
            "system",
            "You are a helpful assistant that translates"
            "{input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)

AIMessage(content='"Ich liebe das Programmieren."', additional_kwargs={'reasoning_content': 'Okay, the user wants to translate "I love programming." into German. Let me think. The verb "love" is "lieben" or "mögen" in German, but "lieben" is more like "love" in the strong sense. "Programming" would be "Programmierung" or maybe "Programmieren". Since the sentence is "I love programming," the structure in German would use the accusative case because "programmieren" is a verb here, but when using it as a noun, it\'s "die Programmierung". \n\nSo the sentence structure would be "Ich liebe die Programmierung." Alternatively, sometimes in German, they might use the infinitive with "lieben", like "Ich liebe zu programmieren," but I think that\'s less common. The more natural way is probably using the noun form. Let me double-check. \n\nWait, actually, "lieben" is transitive, so it takes a direct object. So "die Programmierung" is correct. Another way could be "Ich liebe es, zu programmieren." 

## Tool Calling
ChatQwQ supports tool calling API that lets you describe tools and their arguments, and have the model return a JSON object with a tool to invoke and the inputs to that tool.

### Use with `bind_tools`

In [5]:
from langchain_qwq import ChatQwQ
from langchain_core.tools import tool


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int


llm = ChatQwQ()

llm_with_tools = llm.bind_tools([multiply])

msg = llm_with_tools.invoke("What's 5 times forty two")

print(msg)

content='' additional_kwargs={'reasoning_content': 'Okay, the user is asking "What\'s 5 times forty two". Let me break this down.\n\nFirst, I need to recognize the numbers here. "5" is straightforward, but "forty two" is written out. I should convert that to 42. So the problem is 5 multiplied by 42.\n\nLooking at the tools provided, there\'s a function called multiply that takes two integers. The parameters are first_int and second_int. So I need to pass 5 and 42 to this function.\n\nWait, the order might matter, but multiplication is commutative, so 5 * 42 is the same as 42 * 5. Either way, the result will be the same. \n\nI should structure the tool call with the two integers. Let me make sure I didn\'t misinterpret the numbers. The user wrote forty two, which is definitely 42. So first_int is 5, second_int is 42. \n\nI need to output the tool_call in JSON within the XML tags. The function name is "multiply", and the arguments should be {"first_int":5, "second_int":42}. \n\nDouble-ch

## API reference

For detailed documentation of all ChatQwQ features and configurations head to the [API reference](https://python.langchain.com/api_reference/langchain-qwq/chat_models/langchain_qwq.chat_models.ChatQwQ.html)